In [0]:
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType
data = [
    (("James","","Smith"),["Java","Scala","C++"],"OH","M"),
    (("Anna","Rose",""),["Spark","Java","C++"],"NY","F"),
    (("Julia","","Williams"),["CSharp","VB"],"OH","F"),
    (("Maria","Anne","Jones"),["CSharp","VB"],"NY","M"),
    (("Jen","Mary","Brown"),["CSharp","VB"],"NY","M"),
    (("Mike","Mary","Williams"),["Python","VB"],"OH","M")
 ]
        
schema = StructType([
     StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
     ])),
     StructField('languages', ArrayType(StringType()), True),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True)
 ])

df = spark.createDataFrame(data = data, schema = schema)
df.printSchema()
df.show(truncate=False)


root
-- name: struct (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
-- languages: array (nullable = true)
 |-- element: string (containsNull = true)
-- state: string (nullable = true)
-- gender: string (nullable = true)

+----------------------+------------------+-----+------+
name |languages |state|gender|
+----------------------+------------------+-----+------+
{James, , Smith} |[Java, Scala, C++]|OH |M |
{Anna, Rose, } |[Spark, Java, C++]|NY |F |
{Julia, , Williams} |[CSharp, VB] |OH |F |
{Maria, Anne, Jones} |[CSharp, VB] |NY |M |
{Jen, Mary, Brown} |[CSharp, VB] |NY |M |
{Mike, Mary, Williams}|[Python, VB] |OH |M |
+----------------------+------------------+-----+------+

In [0]:
# Using equals condition
df.filter(df.state == "OH").show(truncate=False)

+----------------------+------------------+-----+------+
name |languages |state|gender|
+----------------------+------------------+-----+------+
{James, , Smith} |[Java, Scala, C++]|OH |M |
{Julia, , Williams} |[CSharp, VB] |OH |F |
{Mike, Mary, Williams}|[Python, VB] |OH |M |
+----------------------+------------------+-----+------+

In [0]:
# not equals condition
df.filter(df.state != "OH") \
    .show(truncate=False) 
df.filter(~(df.state == "OH")) \
    .show(truncate=False)

+--------------------+------------------+-----+------+
name |languages |state|gender|
+--------------------+------------------+-----+------+
{Anna, Rose, } |[Spark, Java, C++]|NY |F |
{Maria, Anne, Jones}|[CSharp, VB] |NY |M |
{Jen, Mary, Brown} |[CSharp, VB] |NY |M |
+--------------------+------------------+-----+------+

+--------------------+------------------+-----+------+
name |languages |state|gender|
+--------------------+------------------+-----+------+
{Anna, Rose, } |[Spark, Java, C++]|NY |F |
{Maria, Anne, Jones}|[CSharp, VB] |NY |M |
{Jen, Mary, Brown} |[CSharp, VB] |NY |M |
+--------------------+------------------+-----+------+

In [0]:
#Using SQL col() function
from pyspark.sql.functions import col
df.filter(col("state") == "OH") \
    .show(truncate=False) 

+----------------------+------------------+-----+------+
name |languages |state|gender|
+----------------------+------------------+-----+------+
{James, , Smith} |[Java, Scala, C++]|OH |M |
{Julia, , Williams} |[CSharp, VB] |OH |F |
{Mike, Mary, Williams}|[Python, VB] |OH |M |
+----------------------+------------------+-----+------+

In [0]:
#Using SQL Expression
df.filter("gender == 'M'").show()
#For not equal
df.filter("gender != 'M'").show()
df.filter("gender <> 'M'").show()

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {James, , Smith}|[Java, Scala, C++]| OH| M|
{Maria, Anne, Jones}| [CSharp, VB]| NY| M|
 {Jen, Mary, Brown}| [CSharp, VB]| NY| M|
{Mike, Mary, Will...| [Python, VB]| OH| M|
+--------------------+------------------+-----+------+

+-------------------+------------------+-----+------+
 name| languages|state|gender|
+-------------------+------------------+-----+------+
 {Anna, Rose, }|[Spark, Java, C++]| NY| F|
{Julia, , Williams}| [CSharp, VB]| OH| F|
+-------------------+------------------+-----+------+

+-------------------+------------------+-----+------+
 name| languages|state|gender|
+-------------------+------------------+-----+------+
 {Anna, Rose, }|[Spark, Java, C++]| NY| F|
{Julia, , Williams}| [CSharp, VB]| OH| F|
+-------------------+------------------+-----+------+

In [0]:
#Filter multiple condition
df.filter( (df.state  == "OH") & (df.gender  == "M") ) \
    .show(truncate=False)  

+----------------------+------------------+-----+------+
name |languages |state|gender|
+----------------------+------------------+-----+------+
{James, , Smith} |[Java, Scala, C++]|OH |M |
{Mike, Mary, Williams}|[Python, VB] |OH |M |
+----------------------+------------------+-----+------+

In [0]:
#Filter IS IN List values
li=["OH","CA","DE"]
df.filter(df.state.isin(li)).show()

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {James, , Smith}|[Java, Scala, C++]| OH| M|
 {Julia, , Williams}| [CSharp, VB]| OH| F|
{Mike, Mary, Will...| [Python, VB]| OH| M|
+--------------------+------------------+-----+------+

In [0]:
# Filter NOT IS IN List values
#These show all records with NY (NY is not part of the list)
df.filter(~df.state.isin(li)).show()
df.filter(df.state.isin(li)==False).show()

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {Anna, Rose, }|[Spark, Java, C++]| NY| F|
{Maria, Anne, Jones}| [CSharp, VB]| NY| M|
 {Jen, Mary, Brown}| [CSharp, VB]| NY| M|
+--------------------+------------------+-----+------+

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {Anna, Rose, }|[Spark, Java, C++]| NY| F|
{Maria, Anne, Jones}| [CSharp, VB]| NY| M|
 {Jen, Mary, Brown}| [CSharp, VB]| NY| M|
+--------------------+------------------+-----+------+

In [0]:
# Using startswith
df.filter(df.state.startswith("N")).show()

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {Anna, Rose, }|[Spark, Java, C++]| NY| F|
{Maria, Anne, Jones}| [CSharp, VB]| NY| M|
 {Jen, Mary, Brown}| [CSharp, VB]| NY| M|
+--------------------+------------------+-----+------+

In [0]:
#using endswith
df.filter(df.state.endswith("H")).show()

#contains
df.filter(df.state.contains("H")).show()

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {James, , Smith}|[Java, Scala, C++]| OH| M|
 {Julia, , Williams}| [CSharp, VB]| OH| F|
{Mike, Mary, Will...| [Python, VB]| OH| M|
+--------------------+------------------+-----+------+

+--------------------+------------------+-----+------+
 name| languages|state|gender|
+--------------------+------------------+-----+------+
 {James, , Smith}|[Java, Scala, C++]| OH| M|
 {Julia, , Williams}| [CSharp, VB]| OH| F|
{Mike, Mary, Will...| [Python, VB]| OH| M|
+--------------------+------------------+-----+------+

In [0]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5,"Rames rose")
  ]
df2 = spark.createDataFrame(data = data2, schema = ["id","name"])

# like - SQL LIKE pattern
df2.filter(df2.name.like("%rose%")).show()

+---+----------+
 id| name|
+---+----------+
 5|Rames rose|
+---+----------+

In [0]:
# rlike - SQL RLIKE pattern (LIKE with Regex)
#This check case insensitive
df2.filter(df2.name.rlike("(?i)^*rose$")).show()

+---+------------+
 id| name|
+---+------------+
 2|Michael Rose|
 4| Rames Rose|
 5| Rames rose|
+---+------------+

In [0]:
from pyspark.sql.functions import array_contains
df.filter(array_contains(df.languages,"Java")) \
    .show(truncate=False)

+----------------+------------------+-----+------+
name |languages |state|gender|
+----------------+------------------+-----+------+
{James, , Smith}|[Java, Scala, C++]|OH |M |
{Anna, Rose, } |[Spark, Java, C++]|NY |F |
+----------------+------------------+-----+------+

In [0]:
#Struct condition
df.filter(df.name.lastname == "Williams") \
    .show(truncate=False) 

+----------------------+------------+-----+------+
name |languages |state|gender|
+----------------------+------------+-----+------+
{Julia, , Williams} |[CSharp, VB]|OH |F |
{Mike, Mary, Williams}|[Python, VB]|OH |M |
+----------------------+------------+-----+------+